In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA  # for QA through llm
from langchain.embeddings import HuggingFaceEmbeddings  # for embedding
from langchain.vectorstores import Pinecone # for vector database
import pinecone

from langchain.document_loaders import PyPDFLoader,DirectoryLoader # for documents uploader
from langchain.text_splitter import RecursiveCharacterTextSplitter # for create the chunks
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


d:\Medical-chatbot\env\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
PINECONE_API_KEY = "Pinecone api key"
PINECONE_API_ENV = "pinecone env key"

In [5]:
# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf("D:\Medical-chatbot\data")

In [8]:
# Create text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print('Length of my chunks: ',len(text_chunks))

Length of my chunks:  7020


In [10]:
## downloading embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [12]:
embeddings = download_hugging_face_embeddings()

pytorch_model.bin: 100%|██████████| 90.9M/90.9M [02:57<00:00, 512kB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 4.52kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 14.3kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 480kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 19.9kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 1.68MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 471kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 44.7kB/s]


In [13]:
query_result = embeddings.embed_query('hello world')
print("Length", len(query_result))

Length 384


In [14]:
# Initializing the PineCone
pinecone.init(api_key=PINECONE_API_KEY,
               environment=PINECONE_API_ENV)
index_name = 'medical-chatbot'

# Creating Embeddings for each of the text chunks and storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [15]:
# if we already have an index we load it like this
docsearch=Pinecone.from_existing_index(index_name,embeddings)

query = "what are Allergies"

docs = docsearch.similarity_search(query,k=3)

print('Result',docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drug

In [16]:
docs[0].page_content

"GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's questions.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only Return the helpful answer below and nothing else.
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)